# Ejemplo de uso: Conexión a bases de datos SQL con `sql_connection`

Este notebook demuestra cómo conectarse a una base de datos existente usando el módulo OOP `sql_connection`.
El objetivo es realizar consultas de lectura sobre una base de datos real, sin crear ni modificar tablas.

Base de ejemplo: **toys_and_models.sqlite** (ubicada en `examples/`).

Puedes sobreescribir la ruta usando la variable de entorno `SQL_CONN_EXAMPLE_DB`.

## Requisitos

Instala el paquete en modo editable desde la raíz del proyecto:

```bash
pip install -e .
```

Instala `pandas` si deseas mostrar los resultados como DataFrames:

In [ ]:
from sql_connection import get_connector
import os
try:
    import pandas as pd
except ImportError:
    pd = None

In [ ]:
# Ruta de la base de datos existente (permite override por variable de entorno)
db_path = os.environ.get("SQL_CONN_EXAMPLE_DB", "examples/toys_and_models.sqlite")
db_path = os.path.abspath(db_path)

if not os.path.exists(db_path):
    raise FileNotFoundError(f"No se encontró la base de datos: {db_path}")

# Crear conector y abrir conexión (context manager)
conn = get_connector("sqlite", path=db_path)
with conn:
    print("✅ Conectado a:", conn.dsn_summary())
    print("Ping:", conn.ping())

In [ ]:
# Listar tablas de la base de datos
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"

if pd:
    display(conn.read_sql(query_tables))
else:
    rows = conn.query(query_tables)
    print("Tablas:", rows)

In [ ]:
# Consultar los primeros clientes (solo lectura)
query_customers = "SELECT customerNumber, customerName, country FROM customers LIMIT 10;"

if pd:
    display(conn.read_sql(query_customers))
else:
    rows = conn.query(query_customers)
    for row in rows:
        print(row)

In [ ]:
# Consulta analítica: top 5 países por número de clientes
query_top_countries = """
SELECT country, COUNT(*) AS num_customers
FROM customers
GROUP BY country
ORDER BY num_customers DESC
LIMIT 5;
"""

if pd:
    display(conn.read_sql(query_top_countries))
else:
    rows = conn.query(query_top_countries)
    print("Top países:", rows)

In [ ]:
# Consulta parametrizada (buenas prácticas para evitar inyección)
country = "USA"
sql_param = """
SELECT customerNumber, customerName, country
FROM customers
WHERE country = :country
LIMIT 5;
"""

params = {"country": country}

if pd:
    display(conn.read_sql(sql_param, params=params))
else:
    rows = conn.query(sql_param, params=params)
    for r in rows:
        print(r)

## Conclusiones

- El flujo OOP permite conectar con distintos motores (SQLite, PostgreSQL, Oracle) sin cambiar el código del análisis.
- `read_sql()` simplifica el análisis exploratorio con `pandas`; `query()` es una alternativa ligera sin dependencias.
- Este notebook es una base para integraciones posteriores (pipelines o dashboards), manteniendo **solo lectura**.